In [1]:
import nltk
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\katyc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\katyc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
def data_cleaning(df, col_name): 
    # Remove the tagged labels and word tokenize the sentence
    df[col_name] = df[col_name].apply(lambda x: re.sub("\s\[.*\]|@(\w+)", "", x))
    df[col_name] = df[col_name].apply(lambda x: re.sub("([^0-9A-Za-z\t\.])|(\w+:\/\/\S+)|^rt|http.+?", " ", x))
    df[col_name] = df[col_name].apply(lambda x: re.sub("\s+", " ", x))

    # To lowercase
    df[col_name] = df[col_name].apply(lambda x: x.lower())
    
    # Remove stop words and Lemmatazing/Stemming
    stops = set(stopwords.words("english"))
    stemmer = PorterStemmer()
    # lemmatizer = WordNetLemmatizer()
    def remove_word(essay):
        return " ".join([stemmer.stem(word) for word in essay.split() if word not in (stops)])
        # return " ".join([lemmatizer.lemmatize(word) for word in essay.split() if word not in (stops)])

    df[col_name] = df[col_name].apply(lambda x: remove_word(x))

In [22]:
# get kaggle dataset
df = pd.read_excel("training_set_rel3.xlsx")
df = df.loc[:, :'domain1_score']
df = df.drop(['rater3_domain1'], axis=1)
df = df.rename(columns={"essay_id": "Essay_id", "essay_set": "Essay_set_id", "essay":"Essay"})
df = df.query("Essay_set_id == 1 or Essay_set_id == 2 or Essay_set_id == 7 or Essay_set_id == 8")

In [23]:
#clean dataset
data_cleaning(df, "Essay")
df.to_csv("cleaned_kaggle_essay_set.csv", index=False)